In [13]:
#Apply the Radon-Nikodym principles to analyze anomalies in ITC's daily closing prices and develop a buy and sell strategy

In [14]:
# This output includes all required data to evaluate and analyze the trading strategy.
#You can fine-tune the buy and sell thresholds to optimize the strategy to reach a Sharpe ratio of 1.85.

In [32]:
import yfinance as yf
import pandas as pd
import numpy as np

start='2023-05-19'
end='2024-05-19'
ticker = 'TSLA'

# Fetch the data
stock = yf.download(ticker, start=start, end=end)
closing_prices = stock['Close']

# Compute returns and Radon-Nikodym derivative
returns = closing_prices.pct_change().dropna()
rolling_mean = returns.rolling(window=20).mean()
rolling_std = returns.rolling(window=30).std()
radon_nikodym_derivative = (returns - rolling_mean) / rolling_std

# Set thresholds and generate signals
buy_threshold = -1
sell_threshold = -5
buy_signals = radon_nikodym_derivative < buy_threshold
sell_signals = radon_nikodym_derivative > sell_threshold

signals = pd.DataFrame(index=returns.index)
signals['Returns'] = returns
signals['Buy'] = buy_signals
signals['Sell'] = sell_signals

# Initialize variables for trading simulation
initial_capital = 100000
capital = initial_capital
position = 0
portfolio_value = [initial_capital]
trades = []
max_drawdown = 0

# Simulate trading
for i in signals.index:
    if signals.loc[i, 'Buy'] and capital > 0:
        buy_price = closing_prices.loc[i]
        position = capital / buy_price
        capital = 0
        trades.append({'Buy Date': i, 'Buy Price': buy_price, 'Sell Date': None, 'Sell Price': None,
                       'Change (%)': None, 'Cumulative Profit %': None, 'Portfolio Value': None})
    elif signals.loc[i, 'Sell'] and position > 0:
        sell_price = closing_prices.loc[i]
        capital = position * sell_price
        position = 0
        trades[-1]['Sell Date'] = i
        trades[-1]['Sell Price'] = sell_price
        trades[-1]['Change (%)'] = ((sell_price - trades[-1]['Buy Price']) / trades[-1]['Buy Price']) * 100

    current_portfolio_value = capital + position * closing_prices.loc[i]
    portfolio_value.append(current_portfolio_value)
    if trades:
        trades[-1]['Portfolio Value'] = current_portfolio_value
        trades[-1]['Cumulative Profit %'] = ((current_portfolio_value / initial_capital) - 1) * 100

# Create DataFrame for summary table
summary_table = pd.DataFrame(trades)

# Simulate trading
for i in signals.index:
    if signals.loc[i, 'Buy'] and capital > 0:
        buy_price = closing_prices.loc[i]
        position = capital / buy_price
        capital = 0
    elif signals.loc[i, 'Sell'] and position > 0:
        sell_price = closing_prices.loc[i]
        capital = position * sell_price
        position = 0
    portfolio_value.append(capital + position * closing_prices.loc[i])

# Calculate cumulative profit
cumulative_profit = ((portfolio_value[-1] / initial_capital) - 1) * 100

# Download market data
market_data = yf.download('^GSPC', start=start, end=end)

# Beta calculation
market_returns = market_data['Close'].pct_change().dropna()
cov_matrix = np.cov(returns, market_returns)
cov_stock_market = cov_matrix[0][1]
var_market = cov_matrix[1][1]
beta = cov_stock_market / var_market

# Calculate max drawdown
portfolio_value_series = pd.Series(portfolio_value)
rolling_max = portfolio_value_series.cummax()
daily_drawdown = portfolio_value_series / rolling_max - 1
max_drawdown = daily_drawdown.min() * 100

# Calculate Sharpe Ratio
risk_free_rate = 0.03
portfolio_returns = pd.Series(portfolio_value).pct_change().dropna()
sharpe_ratio = ((portfolio_returns.mean() - risk_free_rate / 252) / portfolio_returns.std()) * np.sqrt(252)

# Calculate VAR (5th percentile as VAR for 95% confidence)
var = np.percentile(portfolio_returns.dropna() * 100, 5)  # Adjusted for clearer interpretation

# Alpha calculation with the computed beta
benchmark_return = 0.05
alpha = portfolio_returns.mean() - (risk_free_rate + beta * (benchmark_return - risk_free_rate))
print(
    f'Cumulative Profit %: {cumulative_profit:.2f}%, '
    f'Max Drawdown: {max_drawdown:.2f}%, '
    f'Sharpe Ratio: {sharpe_ratio:.2f}, '
    f'VAR: {var:.2f}%, '
    f'Alpha: {alpha:.2f}'
)
round(summary_table,2)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Cumulative Profit %: -19.72%, Max Drawdown: -23.86%, Sharpe Ratio: -0.88, VAR: 0.00%, Alpha: -0.07


,Buy Date,Buy Price,Sell Date,Sell Price,Change (%),Cumulative Profit %,Portfolio Value
0,2023-07-06,276.54,2023-07-07,274.43,-0.76,-0.76,99236.99
1,2023-07-20,262.90,2023-07-21,260.02,-1.10,-1.85,98149.88
2,2023-09-01,245.01,2023-09-05,256.49,4.69,2.75,102748.71
3,2023-09-18,265.28,2023-09-19,266.50,0.46,3.22,103221.24
4,2023-09-22,244.88,2023-09-25,246.99,0.86,4.11,104110.65
5,2023-10-18,242.68,2023-10-19,220.11,-9.30,-5.57,94428.03
6,2023-10-30,197.36,2023-10-31,200.84,1.76,-3.91,96093.06
7,2023-11-09,209.98,2023-11-10,214.65,2.22,-1.77,98230.19
8,2023-11-16,233.59,2023-11-17,234.30,0.30,-1.47,98528.76
9,2023-12-20,247.14,2023-12-21,254.50,2.98,1.46,101463.02
